In [8]:
%bash

ls

UsageError: Line magic function `%bash` not found (But cell magic `%%bash` exists, did you mean that instead?).


In [9]:
! ls 

README.md       Untitled.ipynb  Untitled1.ipynb


In [16]:
ls 

README.md              test/
analysis_script.ipynb  test_script.ipynb


In [24]:
mkdir -p test/
ls

SyntaxError: invalid syntax (1246378638.py, line 1)

In [21]:
rm -r test/

In [22]:
ls

README.md              analysis_script.ipynb  test_script.ipynb
